<a href="https://colab.research.google.com/github/ChristyGKrishnan/ensemble-be/blob/main/Stable_Diff_Inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask
!pip install flask-ngrok2

In [ ]:
!pip install diffusers["torch"] transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 105.8 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.21.1-py3-none-any.whl size=1486516 sha256=e55378499085e16da826c427f0fba3530a2fb9b4ad5d889e8515c14de8ace417
  Stored in directory: /root/.cache/pip/wheels/a8/52/ec/901d7f3e1128bc56ea4365e9d3a157c1088e5450f4e8f7b202
Successfully built diffusers


In [ ]:
from PIL import Image
import base64
from io import BytesIO


def img2base64(image: Image):
    buffered = BytesIO()
    image.save(buffered, format="JPEG")
    return "data:image/jpeg;base64," + base64.b64encode(buffered.getvalue()).decode()

def trim_data(s: str):
  return s[s.find(",") + 1:]

def base642img(image: str):
  image = trim_data(image)
  res = bytes(image, 'utf-8')
  im_bytes = base64.b64decode(res)
  im_file = BytesIO(im_bytes)
  img = Image.open(im_file)

  return img

In [ ]:
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image
import torch

pipe = AutoPipelineForInpainting.from_pretrained("diffusers/stable-diffusion-xl-1.0-inpainting-0.1", torch_dtype=torch.float16, variant="fp16").to("cuda")
generator = torch.Generator(device="cuda").manual_seed(0)

def f(prompt, img, mask):

  image = base642img(img).resize((1024, 1024))
  mask_image = base642img(mask).resize((1024, 1024))

  image = pipe(
  prompt=prompt,
  image=image,
  mask_image=mask_image,
  guidance_scale=8.0,
  num_inference_steps=20,
  strength=0.99,
  generator=generator,
  ).images[0]

  return image

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Fetching 18 files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 37000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [ ]:
from flask import *
from flask_ngrok2 import run_with_ngrok
app = Flask(__name__)
@app.route('/', methods=['POST'])
def home():
  image = f(request.json['text'],request.json['image'],request.json['mask'])
  return {'image' : img2base64(image)}

In [ ]:
run_with_ngrok(app, auth_token="25vfaRMJjAAUpvsjmHyrjwTgkHJ_7YqgDQcwujd2xZKJbczKj")
app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://440d-34-148-89-226.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


  0%|          | 0/19 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2023 05:24:29] "POST / HTTP/1.1" 200 -


...

INFO:werkzeug:127.0.0.1 - - [15/Sep/2023 08:08:34] "POST / HTTP/1.1" 200 -


  0%|          | 0/19 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2023 08:09:07] "POST / HTTP/1.1" 200 -


  0%|          | 0/19 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [15/Sep/2023 08:13:18] "POST / HTTP/1.1" 200 -
